In [2]:
import _init_paths
import tensorflow as tf
from dataset.Dummy import Dummy
from models.CRG import CRG
from models.MMG import MMG
from models.RRG import RRG

##### Test forward pass with dummy dataset

In [3]:
# Dataset parameters
n_samples = 16
n_nodes = 12
node_feat_dim = 8
edge_feat_dim = 4

In [4]:
# Dataset
dummy = Dummy(n_samples, n_nodes, node_feat_dim, edge_feat_dim)
dummy_dataset = dummy._get_db()
adjacency = dummy_dataset['adjacency']
node_features = dummy_dataset['node_features']
edge_features = dummy_dataset['edge_features']
coordinates = dummy_dataset['coordinates']
center_scores = dummy_dataset['center_scores']
joint_types = dummy_dataset['joint_types']
print('adjaceny shape:', adjacency.shape)
print('node features shape:', node_features.shape)
print('edge features shape:', edge_features.shape)
print('coordinates shape:', coordinates.shape)
print('center scores shape:', center_scores.shape)
print('joint types  shape:', joint_types.shape)

adjaceny shape: (16, 12, 12)
node features shape: (16, 12, 8)
edge features shape: (16, 12, 12, 4)
coordinates shape: (16, 12, 3)
center scores shape: (16, 12, 1)
joint types  shape: (16, 12, 15)


In [5]:
# Architecture parameters
# MMG
mmg_hidden_dim = [256, 128, 64]
mmg_output_dim = 1
# CRG
crg_mlp_dim =  [256, 256, 256]
crg_hidden_dim = [32, 32, 128]
crg_output_dim = 1
# RRG
rrg_mlp_dim = [512, 1024, 256, 256, 256]
rrg_hidden_dim = [32, 32, 512, 128, 128]
rrg_output_dim = [3, 1]


In [7]:
# Architecture Init
mmg_model = MMG(mmg_hidden_dim, mmg_output_dim)
crg_model = CRG(crg_mlp_dim, crg_hidden_dim, crg_output_dim)
#rrg_model = RRG(rrg_mlp_dim, rrg_hidden_dim, rrg_output_dim)

In [8]:
# Forward Pass
# MMG
mmg_output = mmg_model(adjacency, node_features, edge_features)
output_graphs = tf.reshape(mmg_output, adjacency.shape)
print(mmg_output.shape)
print(output_graphs.shape)
print(output_graphs)

(16, 12, 12)
(16, 12, 12)
tf.Tensor(
[[[0.49993828 0.510495   0.50846714 ... 0.5057817  0.5068571  0.50308305]
  [0.5036193  0.5133131  0.51352936 ... 0.5102502  0.5121088  0.50747216]
  [0.49928814 0.5077107  0.5074461  ... 0.50467527 0.50615984 0.50339735]
  ...
  [0.5018156  0.5112687  0.5105842  ... 0.5079068  0.5096881  0.5058505 ]
  [0.5016655  0.5098252  0.5096593  ... 0.5058663  0.50736564 0.50339943]
  [0.4954424  0.5022747  0.5027039  ... 0.49930978 0.5018141  0.4971949 ]]

 [[0.5014205  0.5021823  0.50315773 ... 0.49484026 0.49915767 0.5020927 ]
  [0.5036763  0.5040846  0.50562805 ... 0.49825925 0.50334406 0.5062331 ]
  [0.50448596 0.50473017 0.506808   ... 0.49861753 0.5038445  0.50726575]
  ...
  [0.5074161  0.5070909  0.5095246  ... 0.5005175  0.50475085 0.5082704 ]
  [0.50670046 0.5070573  0.50865126 ... 0.5009067  0.5051628  0.50778097]
  [0.50878024 0.5067932  0.50822747 ... 0.49957946 0.5044086  0.50817055]]

 [[0.5068977  0.5091038  0.5064744  ... 0.5094071  0.507782

In [11]:
# train_step 
# j'ai mis la sortie en target juste pr être sur d'avoir les bnes dimensions
target = mmg_output
mmg_model.train_step([adjacency, node_features, edge_features], target )


In [9]:
# CRG
crg_output = crg_model(coordinates, adjacency, node_features, center_scores)
print(crg_output.shape)

(16, 12, 1)


In [ ]:
# RRG
rrg_output1, rrg_output2 = rrg_model(coordinates, adjacency, node_features, edge_features, joint_types)
print(rrg_output1.shape)
print(rrg_output2.shape)

ValueError: Exception encountered when calling layer "edge_conv_3" (type EdgeConv).

neighbors must have a rank of 2, but it has rank 3 and shape [16, 12, 12]

Call arguments received:
  • Adjacency=<tensorflow.python.framework.sparse_tensor.SparseTensor object at 0x000001F7CEEE7488>
  • node_features=tf.Tensor(shape=(16, 512), dtype=float32)